Imports

In [312]:
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import open3d as o3d
import pandas as pd
import regex
import copy

Point Cloud Visualization

In [313]:
def cleanTxt(oldFile, i):
    # Takes in a txt file and with pandas saves a txt file that can
    # be used by open3d
    data = pd.read_csv(oldFile, header=None)    
    np.savetxt(f'FinalPointClouds/pointCloud{i}', data.values)
    
    

In [314]:
def visualizePointCloud(file):
    # Takes a point cloud file to load and visualize
    pointCloud = file

    # Takes the point cloud data and visualizes it
    pcd = o3d.io.read_point_cloud(pointCloud, format='xyzrgb')
    
    #print(len(np.asarray(pcd.points)))
    o3d.visualization.draw_geometries([pcd])

Receive txt files from Unity and send to output directory

In [315]:
def textFileCleanup():
    # clean output directory first
    # convert unity txt to readable txt
    # save each point cloud txt in final directory
    
    # Paths of relevant directories
    unityPCs = 'My project/Assets/PointClouds'
    cleanPCs = 'FinalPointClouds'

    # Clean output directory
    for fileName in os.listdir(cleanPCs):
        file = os.path.join(cleanPCs, fileName)
        
        if os.path.isfile(file):
            print('Deleting file:', file)
            os.remove(file)
            
    # For every txt file outputted by Unity, save to output directory
    for f in os.listdir(unityPCs):
        unityFile = os.path.join(unityPCs, f)

        if os.path.isfile(unityFile) and unityFile.endswith('.txt'):
            # use regex to know which number file we are on and send to clean
            index = int(regex.findall(r'\d+', unityFile)[0])
            cleanTxt(unityFile, index)
            
        
textFileCleanup()



Deleting file: FinalPointClouds\pointCloud0
Deleting file: FinalPointClouds\pointCloud1
Deleting file: FinalPointClouds\pointCloud2
Deleting file: FinalPointClouds\pointCloud3
Deleting file: FinalPointClouds\pointCloud4
Deleting file: FinalPointClouds\pointCloud5
Deleting file: FinalPointClouds\pointCloud6
Deleting file: FinalPointClouds\pointCloud7
Deleting file: FinalPointClouds\pointCloud8
Deleting file: FinalPointClouds\pointCloud9


In [316]:
def sortPointCloudFiles(listOfFiles):
    # Takes a list of txt point cloud files that are numbered, sorts and 
    # then returns the sorted list
    
    # Initialize an empty list to insert values into
    sortedList = ["" for x in range(len(listOfFiles))]
    
    for file in listOfFiles:
        # Find index associated to txt file and add to that position in
        # sorted list
        index = int(regex.findall(r'\d+', file)[0])
        sortedList[index] = file
    
    return sortedList

ICP Registration

In [317]:
def draw_registration_result(source, target, transformation):
    # A helper function that visualizes the transformed source point cloud
    # with the target point cloud. Provided from the Open3D documentation.
    
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0]) #Yellowish cloud
    target_temp.paint_uniform_color([0, 0.651, 0.929]) #bluish cloud
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

def ICP(pointCloud1, pointCloud2, pc1Form, pc2Form):
    # Receives as input two point clouds, and merges the 2nd point cloud
    # into the first with ICP. The pcForm condition refers to if pc1 or pc2 is 
    # already in .ply format or not
    
    if(pc1Form == True):
        pc1 = o3d.io.read_point_cloud(pointCloud1)
    else: pc1 = o3d.io.read_point_cloud(pointCloud1, format='xyzrgb') 
    
    if(pc2Form == True):
        pc2 = o3d.io.read_point_cloud(pointCloud2)
    else: pc2 = o3d.io.read_point_cloud(pointCloud2, format='xyzrgb')
    
    # ICP parameters
    threshold = 0.02
    trans_init = np.eye(4)
    
    # ICP registration pipeline
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source=pc1, target=pc2, max_correspondence_distance=threshold, init=trans_init,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
    
    # Visualize ICP
    #draw_registration_result(pc1, pc2, reg_p2p.transformation)
    
    return reg_p2p
    
def mergePointClouds():
    # For every point cloud in the FinalPointClouds directory, create a 
    # final point cloud by merging them all with ICP
    finalPC = o3d.geometry.PointCloud()
    print('Type of finalPC:', type(finalPC))

    # list of txt files in FinalPointClouds directory
    pointCloudsDir = 'FinalPointClouds'
    onlyfiles = [f for f in os.listdir(pointCloudsDir) if os.path.isfile(join(pointCloudsDir, f))]
    
    sortedFiles = sortPointCloudFiles(onlyfiles)
    
    # Set a starting pointcloud txt file
    firstPC = sortedFiles[0]
    
    for i in range(1, len(sortedFiles)):
        # For every subsequent point cloud file, merge to the first one
        target = sortedFiles[i]
        print('Merging: ', firstPC, target)
        
        # Iterate over all other txt files and use ICP to merge into one point cloud
        if(i == 1):
            # If its the first iteration, the point cloud is any arbitrary point cloud
            finalMerge = False
            pathPC1 = pointCloudsDir + '/' + firstPC    
            pcd1 = o3d.io.read_point_cloud(pathPC1, format='xyzrgb')
        
        else:
            # On all other iterations, we want to merge with the final cloud 
            finalMerge = True
            pathPC1 = 'mergedPC_run18.ply'
            pcd1 = o3d.io.read_point_cloud(pathPC1)
        
        pathPC2 = pointCloudsDir + '/' + target
        pcd2 = o3d.io.read_point_cloud(pathPC2, format='xyzrgb')
        
        # Use ICP to find transformation matrix between the two
        transformationInfo = ICP(pathPC1, pathPC2, finalMerge, False)
        transformationMatrix = transformationInfo.transformation
        
        finalPC = pcd1.transform(transformationMatrix)
        finalPC += pcd2.transform(transformationMatrix)
        
        o3d.io.write_point_cloud('mergedPC_run18.ply', finalPC, write_ascii=True)
            
    return finalPC

mergedPointCloud = mergePointClouds()


Type of finalPC: <class 'open3d.cpu.pybind.geometry.PointCloud'>
Merging:  pointCloud0 pointCloud1
Merging:  pointCloud0 pointCloud2
Merging:  pointCloud0 pointCloud3
Merging:  pointCloud0 pointCloud4
Merging:  pointCloud0 pointCloud5
Merging:  pointCloud0 pointCloud6
Merging:  pointCloud0 pointCloud7
Merging:  pointCloud0 pointCloud8
Merging:  pointCloud0 pointCloud9
Merging:  pointCloud0 pointCloud10
Merging:  pointCloud0 pointCloud11
Merging:  pointCloud0 pointCloud12
Merging:  pointCloud0 pointCloud13
Merging:  pointCloud0 pointCloud14


In [318]:
 # Takes the final point cloud data and visualizes it
final = o3d.io.read_point_cloud('mergedPC_run18.ply')

print(len(np.asarray(final.points)))
o3d.visualization.draw_geometries([final])

140976


In [323]:
def mergeFinalClouds(finalCloud1, finalCloud2):
    # takes two complete clouds that are a combination of the original point 
    # clouds registered, and merge them into one 
    finalPC = o3d.geometry.PointCloud()
    
    # Use ICP to find transformation matrix between the two
    transformationInfo = ICP(finalCloud1, finalCloud2, True, True)
    transformationMatrix = transformationInfo.transformation
    
    pcd1 = o3d.io.read_point_cloud(finalCloud1)
    pcd2 = o3d.io.read_point_cloud(finalCloud2)
    
    finalPC = pcd1.transform(transformationMatrix)
    finalPC += pcd2.transform(transformationMatrix)
    
    o3d.io.write_point_cloud('mergedTest_add18.ply', finalPC, write_ascii=True)
        
    return finalPC


#Testing/Visualizing purposes

#mergeTest = mergeFinalClouds('mergedTest_add17.ply', 'mergedPC_run18.ply')
#mergeTest = o3d.io.read_point_cloud('mergedTest_add18.ply')
#print(len(np.asarray(mergeTest.points)))
#o3d.visualization.draw_geometries([mergeTest])




After iteratively merging point clouds as they were processed, the final cloud is composed of about 200 point clouds that have 10,000 points each. The final cloud is as follows:

In [327]:
finalMergedPointCloud = o3d.io.read_point_cloud('FINALCLOUD.ply')
print(f'Cloud has {len(np.asarray(finalMergedPointCloud.points))} total points in it')
o3d.visualization.draw_geometries([finalMergedPointCloud])

Cloud has 2029947 total points in it
